In [1]:
import numpy as np
import time
import copy
import sys
print(sys.path)
sys.path.append('/Users/ziniuwu/Desktop/research/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.pgmpy_BN import Pgmpy_BN
from Evaluation.utils import parse_query

['/Users/ziniuwu/Desktop/research/BayesCard/Testing', '/Users/ziniuwu/anaconda3/envs/bayesnet/lib/python37.zip', '/Users/ziniuwu/anaconda3/envs/bayesnet/lib/python3.7', '/Users/ziniuwu/anaconda3/envs/bayesnet/lib/python3.7/lib-dynload', '', '/Users/ziniuwu/anaconda3/envs/bayesnet/lib/python3.7/site-packages', '/Users/ziniuwu/anaconda3/envs/bayesnet/lib/python3.7/site-packages/pgmpy-0.1.10.dev0-py3.7.egg', '/Users/ziniuwu/anaconda3/envs/bayesnet/lib/python3.7/site-packages/IPython/extensions', '/Users/ziniuwu/.ipython']


ModuleNotFoundError: No module named 'DataPrepare'

In [ ]:
from Schemas.imdb.schema import gen_job_light_imdb_schema
schema = gen_job_light_imdb_schema('/home/ziniu.wzn/imdb-benchmark')

In [121]:
import time
import bz2
import pickle
import logging
import ast

from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.pgmpy_BN import Pgmpy_BN

logger = logging.getLogger(__name__)

OPS = {
    '>': np.greater,
    '<': np.less,
    '>=': np.greater_equal,
    '<=': np.less_equal,
    '=': np.equal,
    '==': np.equal
}

class BN_ensemble():
    """
    Several BNs combined one for each table.
    """

    def __init__(self, schema_graph, bns=dict()):
        self.schema_graph = schema_graph
        self.bns = bns
        self.cached_expectation_vals = dict()
        self.join_size = dict()
        self.join_prepare = None

    def add_BN(self, bn):
        self.bns[bn.table_name] = bn

    def save(self, path, compress=False):
        if compress:
            with bz2.BZ2File(path, 'wb') as f:
                pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)
        else:
            with open(path, 'wb') as f:
                pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)

    def build_from_data(self, hdf_path, ensemble_path, max_table_data, sample_size=None, schema=None):
        """
        We will learn one BN each for each table in the schema graph
        Parameters
        ----------
        hdf_path: path to the folder that contain all pandas dataframe
        ensemble_path: path to save the learned BN ensemble
        max_table_data: max row per hdf file
        sample_size: How many sample to learning BN, if None then use the full data
        schema: containing all information about the graph and attributes
        """
        meta_data_path = hdf_path + '/meta_data.pkl'
        prep = JoinDataPreparator(meta_data_path, schema, max_table_data=max_table_data)
        self.join_prepare = prep

        if schema is not None:
            self.schema_graph = schema

        logger.info(f"Creating a BN for every relationship.")
        for relationship_obj in schema.relationships:
            logger.info(f"Learning BN for {relationship_obj.identifier}.")
            df_samples, meta_types, null_values, full_join_est = prep.generate_n_samples(
                sample_size, relationship_list=[relationship_obj.identifier])
            logger.debug(f"Requested {sample_size} samples and got {len(df_samples)}")
            bn = Pgmpy_BN()
            bn.build_from_data(df_samples)
            self.add_BN(bn)

        ensemble_path += '/ensemble_relationships_' + str(sample_size) + '.pkl'
        logger.info(f"Saving ensemble to {ensemble_path}")
        self.save(ensemble_path)

    def str_pattern_matching(self, s):
        # split the string "attr==value" to ('attr', '=', 'value')
        op_start = 0
        if len(s.split(' IN ')) != 1:
            s = s.split(' IN ')
            attr = s[0].strip()
            try:
                value = list(ast.literal_eval(s[1].strip()))
            except:
                value = s[1].strip()[1:][:-1].split(',')
            return attr, 'in', value

        for i in range(len(s)):
            if s[i] in OPS:
                op_start = i
                if i + 1 < len(s) and s[i + 1] in OPS:
                    op_end = i + 1
                else:
                    op_end = i
                break
        attr = s[:op_start]
        value = s[(op_end+1):]
        ops = s[op_start:(op_end+1)]
        try:
            value = list(ast.literal_eval(s[1].strip()))
        except:
            try:
                value = float(value)
            except:
                value = value
        
        return attr, ops, value
    
    def construct_table_query(self, table_name, table_query, attr, ops, val, epsilon=1e-6):
        query_domain = []
        if self.bns[table_name] is None or attr not in self.bns[table_name].attr_type:
            return None
        if self.bns[table_name].attr_type[attr] == 'continuous':
            if ops == ">=": query_domain = [val, np.infty]
            elif ops == ">": query_domain = [val+epsilon, np.infty]
            elif ops == "<=": query_domain = [-np.infty, val]
            elif ops == "<": query_domain = [-np.infty, val-epsilon]
            elif ops == "=" or ops == "==": query_domain = [val, val]
            else:
                assert False, f"operation {ops} is invalid for continous domain"
                
            if attr not in table_query:
                table_query[attr] = query_domain
            else:
                prev_l = table_query[attr][0]
                prev_r = table_query[attr][1]
                query_domain = [max(prev_l, query_domain[0]), min(prev_r, query_domain[1])]
                table_query[attr] = query_domain
            
        else:
            attr_domain = self.bns[table_name].domain[attr]
            if type(attr_domain[0]) != str:
                attr_domain = np.asarray(attr_domain)
            if ops == "in": 
                assert type(val) == list, "use list for in query"
                query_domain = val
            elif ops == "=" or ops == "==":
                if type(val) == list:  query_domain = val
                else: query_domain = [val]
            else:
                if type(val) == list:
                    assert len(val) == 1
                    val = val[0]
                    assert (type(val) == int or type(val) == float)
                operater = OPS[ops]
                query_domain = list(attr_domain[operater(attr_domain, val)])
                
            if attr not in table_query:
                table_query[attr] = query_domain
            else:
                query_domain = [i for i in query_domain if i in table_query[attr]]
                table_query[attr] = query_domain
                
        return table_query
            

    def store_join_size(self):
        # Store the table size for each relation in the schema
        for relationship_obj in self.schema_graph.relationships:
            rel = relationship_obj.identifier
            self.join_size[rel] = self.join_prepare._size_estimate(relationship_list=[rel])[1]

    def get_full_join_size(self, rel):
        if type(rel) == list and len(rel) == 1:
            rel = rel[0]
        if type(rel) == str:
            return self.join_size[rel]
        else:
            return self.join_prepare._size_estimate(relationship_list=rel)[1]

    def naive_cardinality(self, query):
        # estimate the cardinality of given query
        select_tables = query.table_set
        conditions = query.table_where_condition_dict
        relations = query.relationship_set
        print(select_tables)
        print(conditions)
        print(relations)
        # reformulate them to Single_BN executable form
        p_estimate = 1
        for table_name in select_tables:
            print(table_name)
            print(conditions.keys())
            if table_name in conditions:
                print(table_name)
                table_query = dict()
                for condition in conditions[table_name]:
                    attr, ops, val = self.str_pattern_matching(condition)
                    #attr = table_name + '.' + attr
                    table_query = self.construct_table_query(table_name, table_query, attr, ops, val)
                    print(table_query)
                    if table_query is None:
                        return None
                p = self.bns[table_name].query(table_query, return_prob=True)
                print(p[0])
                p_estimate *= p[0]
        print(self.get_full_join_size(list(relations)))
        return p_estimate * self.get_full_join_size(list(relations))

In [122]:
def read_table_csv(table_obj):
    """
    Reads csv from path, renames columns and drops unnecessary columns
    """
    csv_file_location = table_obj.csv_file_location + "/" + table_obj.table_name + ".csv"
    df_rows = pd.read_csv(csv_file_location, header=None, escapechar='\\', encoding='utf-8', quotechar='"',
                          sep=',')
    df_rows.columns = table_obj.attributes

    for attribute in table_obj.irrelevant_attributes:
        df_rows = df_rows.drop(attribute, axis=1)
    return df_rows.apply(pd.to_numeric, errors="ignore")

def train_all_single():
    for table_obj in schema.tables:
        df = read_table_csv(table_obj)
        for col in df.columns:
            if df[col].nunique() > 1000:
                df = df.drop(col, axis=1)
            else:
                print(col, df[col].nunique())
        if len(df) > 10000000:
            df = df.sample(10000000)
        if len(df.columns) == 0:
            continue
        for algorithm in ["chow-liu", "greedy", "exact", "junction"]:
            print(f'{table_obj.table_name}_{algorithm}')
            BN = Pgmpy_BN(table_obj.table_name)
            BN.build_from_data(df, algorithm=algorithm, max_parents=3, ignore_cols=['id'], sample_size=200000)
            pickle.dump(BN, open(f'{table_obj.table_name}_{algorithm}.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

def load_ensemble(schema, algorithm):
    BN_e= BN_ensemble(schema)
    for table_obj in schema.tables:
        file = "check_points/" + table_obj.table_name + "_" + algorithm + ".pkl"
        try:
            with open(file, 'rb') as f:
                BN = pickle.load(f)
            BN_e.add_BN(BN)
        except:
            BN_e.bns[table_obj.table_name] = None
    meta_data_path = '/home/ziniu.wzn/imdb-benchmark/gen_single_light/meta_data.pkl'
    prep = JoinDataPreparator(meta_data_path, schema, max_table_data=100000000)
    BN_e.join_prepare = prep
    BN_e.store_join_size()
    return BN_e

In [123]:
from time import perf_counter
def evaluate_cardinality(BN_e, schema, query_path, true_cardinalities_path):
    # read all queries
    with open(query_path) as f:
        queries = f.readlines()
    df_true_card = pd.read_csv(true_cardinalities_path)
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):

        print(f"Predicting cardinality for query {query_no}: {query_str}")

        query = parse_query(query_str.strip(), schema)
        cardinality_true = df_true_card.loc[df_true_card['query_no'] == query_no, ['cardinality_true']].values[0][0]
        card_start_t = perf_counter()
        cardinality_predict = BN_e.naive_cardinality(query)
        if cardinality_predict is None:
            continue
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif cardinality_predict == 0:
            cardinality_predict = 1
        elif cardinality_true == 0:
            cardinality_true = 1

        q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        print(f"latency: {latency_ms} and error: {q_error}")
        latencies.append(latency_ms)
        q_errors.append(q_error)
    return latencies, q_errors

In [124]:
true_cardinalities_path = '/home/ziniu.wzn/deepdb-public/benchmarks/job-light/sql/job_light_true_cardinalities.csv'
query_path = '/home/ziniu.wzn/deepdb-public/benchmarks/job-light/sql/job_light_queries.sql'

In [125]:
BN_e = load_ensemble(schema, "chow-liu")

In [80]:
query = "SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=113 AND mc.company_type_id=2 AND t.production_year>2005 AND t.production_year<2010"
query = parse_query(query, schema)

In [126]:
latencies, q_errors = evaluate_cardinality(BN_e, schema, query_path, true_cardinalities_path)

Predicting cardinality for query 0: SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2;

{'movie_companies', 'title', 'movie_info_idx'}
{'movie_info_idx': ['info_type_id=112'], 'movie_companies': ['company_type_id=2']}
{'movie_companies.movie_id = title.id', 'movie_info_idx.movie_id = title.id'}
movie_companies
dict_keys(['movie_info_idx', 'movie_companies'])
movie_companies
{'company_type_id': [2.0]}
0.5116201613641947
title
dict_keys(['movie_info_idx', 'movie_companies'])
movie_info_idx
dict_keys(['movie_info_idx', 'movie_companies'])
movie_info_idx
{'info_type_id': [112.0]}
0.00018115482578340405
5524166.33173042
latency: 3.256041556596756 and error: 1.3965025630860004
Predicting cardinality for query 1: SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=113 AND mc.company_ty

0.7120832564875172
23861233.05487416
latency: 12.840338051319122 and error: 1.4653712487912767
Predicting cardinality for query 14: SELECT COUNT(*) FROM movie_keyword mk,title t,cast_info ci WHERE t.id=mk.movie_id AND t.id=ci.movie_id AND t.production_year>2010 AND mk.keyword_id=8200;

{'cast_info', 'title', 'movie_keyword'}
{'title': ['production_year>2010'], 'movie_keyword': ['keyword_id=8200']}
{'movie_keyword.movie_id = title.id', 'cast_info.movie_id = title.id'}
cast_info
dict_keys(['title', 'movie_keyword'])
title
dict_keys(['title', 'movie_keyword'])
title
{'production_year': [2012.0, 2011.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2019.0]}
0.15945903802473482
movie_keyword
dict_keys(['title', 'movie_keyword'])
movie_keyword
None
Predicting cardinality for query 15: SELECT COUNT(*) FROM movie_keyword mk,title t,cast_info ci WHERE t.id=mk.movie_id AND t.id=ci.movie_id AND t.production_year>2014;

{'cast_info', 'title', 'movie_keyword'}
{'title': ['production_year>2014']}
{'movie_

{'movie_companies.movie_id = title.id', 'movie_keyword.movie_id = title.id'}
movie_keyword
dict_keys(['movie_keyword', 'movie_companies'])
movie_keyword
None
Predicting cardinality for query 32: SELECT COUNT(*) FROM title t,movie_keyword mk,movie_companies mc WHERE t.id=mk.movie_id AND t.id=mc.movie_id AND t.production_year>1950;

{'movie_keyword', 'movie_companies', 'title'}
{'title': ['production_year>1950']}
{'movie_companies.movie_id = title.id', 'movie_keyword.movie_id = title.id'}
movie_keyword
dict_keys(['title'])
movie_companies
dict_keys(['title'])
title
dict_keys(['title'])
title
{'production_year': [1980.0, 2010.0, 1962.0, 2012.0, 1971.0, 1997.0, 2005.0, 2011.0, 2008.0, 1966.0, 1998.0, 2009.0, 1951.0, 2004.0, 2006.0, 1977.0, 2002.0, 1995.0, 1990.0, 2003.0, 1992.0, 1956.0, 1994.0, 1989.0, 1999.0, 1982.0, 1993.0, 2001.0, 1986.0, 2007.0, 1991.0, 2000.0, 1996.0, 1967.0, 1987.0, 1981.0, 1970.0, 1964.0, 2013.0, 1968.0, 1983.0, 1984.0, 1959.0, 1979.0, 1988.0, 1961.0, 1976.0, 1972.0

dict_keys(['movie_info', 'title'])
movie_info
{'info_type_id': [16.0]}
0.2046013
movie_keyword
dict_keys(['movie_info', 'title'])
movie_companies
dict_keys(['movie_info', 'title'])
title
dict_keys(['movie_info', 'title'])
title
{'production_year': [2010.0, 2012.0, 1997.0, 2005.0, 2011.0, 2008.0, 1998.0, 2009.0, 2004.0, 2006.0, 2002.0, 1995.0, 2003.0, 1992.0, 1994.0, 1999.0, 1993.0, 2001.0, 2007.0, 1991.0, 2000.0, 1996.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2019.0]}
0.7120832564875172
62056540.9523058
latency: 14.124300330877304 and error: 69.16138188784866
Predicting cardinality for query 43: SELECT COUNT(*) FROM cast_info ci,title t,movie_keyword mk,movie_companies mc WHERE t.id=ci.movie_id AND t.id=mk.movie_id AND t.id=mc.movie_id AND mk.keyword_id=117;

{'cast_info', 'movie_keyword', 'movie_companies', 'title'}
{'movie_keyword': ['keyword_id=117']}
{'movie_companies.movie_id = title.id', 'movie_keyword.movie_id = title.id', 'cast_info.movie_id = title.id'}
cast_info
dict_keys(['

{'movie_info', 'movie_keyword', 'movie_companies', 'title'}
{'movie_keyword': ['keyword_id=398'], 'movie_companies': ['company_type_id=2'], 'title': ['production_year>1950', 'production_year<2000']}
{'movie_companies.movie_id = title.id', 'movie_info.movie_id = title.id', 'movie_keyword.movie_id = title.id'}
movie_info
dict_keys(['movie_keyword', 'movie_companies', 'title'])
movie_keyword
dict_keys(['movie_keyword', 'movie_companies', 'title'])
movie_keyword
None
Predicting cardinality for query 53: SELECT COUNT(*) FROM title t,movie_keyword mk,movie_companies mc,movie_info mi WHERE t.id=mk.movie_id AND t.id=mc.movie_id AND t.id=mi.movie_id AND mk.keyword_id=398 AND mc.company_type_id=2 AND t.production_year>2000 AND t.production_year<2010;

{'movie_info', 'movie_keyword', 'movie_companies', 'title'}
{'movie_keyword': ['keyword_id=398'], 'movie_companies': ['company_type_id=2'], 'title': ['production_year>2000', 'production_year<2010']}
{'movie_companies.movie_id = title.id', 'movie_in

0.15945903802473482
761537408.269494
latency: 14.950066804885864 and error: 7.733398025298334
Predicting cardinality for query 62: SELECT COUNT(*) FROM title t,cast_info ci,movie_keyword mk,movie_info_idx mi_idx WHERE t.id=mk.movie_id AND t.id=ci.movie_id AND t.id=mi_idx.movie_id AND t.production_year>2000 AND t.kind_id=1 AND mi_idx.info_type_id=101;

{'cast_info', 'movie_keyword', 'title', 'movie_info_idx'}
{'title': ['production_year>2000', 'kind_id=1'], 'movie_info_idx': ['info_type_id=101']}
{'movie_info_idx.movie_id = title.id', 'movie_keyword.movie_id = title.id', 'cast_info.movie_id = title.id'}
cast_info
dict_keys(['title', 'movie_info_idx'])
movie_keyword
dict_keys(['title', 'movie_info_idx'])
title
dict_keys(['title', 'movie_info_idx'])
title
{'production_year': [2010.0, 2012.0, 2005.0, 2011.0, 2008.0, 2009.0, 2004.0, 2006.0, 2002.0, 2003.0, 2001.0, 2007.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2019.0]}
{'production_year': [2010.0, 2012.0, 2005.0, 2011.0, 2008.0, 2009.0, 20

In [129]:
print(np.mean(latencies))
q_errors=np.asarray(q_errors)
for i in [50, 90, 95, 99]:
    print(np.percentile(q_errors, i))

16.189927102199622
2.2685221262265776
28.022603405154452
74.68733190514206
326.9458546088673


In [137]:
for i, obj in enumerate(schema.relationships):
    print(i, obj.identifier)

0 movie_info_idx.movie_id = title.id
1 movie_info.movie_id = title.id
2 cast_info.movie_id = title.id
3 movie_keyword.movie_id = title.id
4 movie_companies.movie_id = title.id


In [131]:
for i, table_obj in enumerate(schema.tables):
    print(i, table_obj.table_name)

0 title
1 movie_info_idx
2 movie_info
3 cast_info
4 movie_keyword
5 movie_companies


In [89]:
df = read_table_csv(schema.tables[4])
for col in ["id", "movie_id"]:
    df = df.drop(col, axis=1)
if len(df) > 10000000:
    df = df.sample(10000000)
df.head(10)

,keyword_id
0,1
1,2
2,2
3,3
4,2
5,4
6,5
7,6
8,7
9,8


In [88]:
df['keyword_id'].nunique()

134170

In [90]:
table_obj = schema.tables[4]
for algorithm in ["chow-liu", "greedy", "exact", "junction"]:
        print(f'{table_obj.table_name}_{algorithm}')
        BN = Pgmpy_BN(table_obj.table_name)
        BN.build_from_data(df, algorithm=algorithm, max_parents=3, ignore_cols=['id'], sample_size=200000)
        pickle.dump(BN, open(f'check_points/{table_obj.table_name}_{algorithm}.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

movie_keyword_chow-liu


KeyboardInterrupt: 

In [19]:
len(df.columns)

0

In [76]:
np.greater(np.ones(6), 2)

array([False, False, False, False, False, False])